# IndoML 2023 Tuturial: Part 2
## The Era of LLMs!

### In-context learning and Prompt Engineering


1. We will use recent LLMs like GPT-3/FLAN-T5/LLAMA to query the models in natural language to get answers/predictions.
2. These models are finetuned on instructions or human-feedbacks to enable them to perform a task through "prompting".
3. Best part is we wouldn't need to train our models to get started, direct inference from these pretrained models is fine.
    * NOTE: Although there can be methods to finetune these models on our data to get better results, we will not be covering that in this tutorial.

### Methods that we will try:

1. FLAN-T5

## Load `dataset`

In [1]:
%env CUDA_VISIBLE_DEVICES=3

env: CUDA_VISIBLE_DEVICES=3


In [2]:
from tqdm import tqdm
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

tqdm.pandas()

dataset = load_dataset("AmazonScience/massive")


/nlsasfs/home/ttbhashini/arroy/anaconda3/envs/py38-new/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load `AutoTokenizer` and `AutoModelForSeq2SeqLM`

In [3]:
# pip install -q transformers accelerate bitsandbytes
from transformers import AutoTokenizer, AutoModelForCausalLM

# checkpoint = "bigscience/mt0-base"
checkpoint = "bigscience/bloomz-3b"
# checkpoint = "google/flan-t5-xxl"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="auto", load_in_8bit=True)

## Model Specific Example of Prompt Engineering

In [8]:
inputs = tokenizer.encode("Detect the intent class of the utterance.\nUtterance: I am going to school.; Intent:", return_tensors="pt").to("cuda")
outputs = model.generate(inputs)
print(tokenizer.decode(outputs[0]))

/nlsasfs/home/ttbhashini/arroy/anaconda3/envs/py38-new/lib/python3.8/site-packages/transformers/generation/utils.py:1254: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/nlsasfs/home/ttbhashini/arroy/anaconda3/envs/py38-new/lib/python3.8/site-packages/transformers/generation/utils.py:1262: UserWarning: Input length of input_ids is 22, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


Detect the intent class of the utterance.
Utterance: I am going to school.; Intent: go


## Preprocessing

- We will create prompts for each test sample in the dataset. 
- There are few ways to format these prompt and this step is called "Prompt Engineering".
    - Few-shot In-context learning: Use task-description and examples
    - Zero-shot In-context learning: Use task-description only

## Exemplars for the multilingual intent-detection task

In [5]:
# Gather examples from the training dataset
import pandas as pd
df_train = dataset['train'].to_pandas()

# Extract one random sample per intent, we will randomly sample the rows
# Hope is that the model will predict english labels for any language that way.
df_intent_samples = df_train.groupby("intent").apply(lambda x: x.sample(1, random_state=42)).reset_index(drop=True)    

In [6]:
# Add formatted prompt for each sample
def int2str(x):
    return dataset['train'].features['intent'].int2str(x)

df_intent_samples['example_prompt_format'] = df_intent_samples.apply(lambda x: f'Utterance: {x["utt"]}; Intent: {int2str(x["intent"])}', axis=1)

# merge examples into a single string
prompt_exemplars = df_intent_samples['example_prompt_format'].str.cat(sep='\n')

In [9]:
print(prompt_exemplars)

Utterance: የዛሬውን ቀን ልትነግሩኝ ትችላላቹ; Intent: datetime_query
Utterance: cambia las luces a azul; Intent: iot_hue_lightchange
Utterance: je veux un billet de train pour l' oregon; Intent: transport_ticket
Utterance: உணவகத்தில் எடுத்துச் செல்லும் வசதி உள்ளதா; Intent: takeaway_query
Utterance: berapakah nilai saham apple; Intent: qa_stock
Utterance: buna; Intent: general_greet
Utterance: olly ni matukio yapi yanafanyika karibu na mimi leo; Intent: recommendation_events
Utterance: bu mahnını xoşlamadığımı yadda saxla; Intent: music_dislikeness
Utterance: oprește-mi priza wemo acum; Intent: iot_wemo_off
Utterance: hvor mange grøntsager burde jeg spise om dagen; Intent: cooking_recipe
Utterance: இந்திய ரூபாவுடன் டாலரின் மாற்று விகிதம் என்ன; Intent: qa_currency
Utterance: 運輸; Intent: transport_traffic
Utterance: چطور پیش می‌رود; Intent: general_quirky
Utterance: آج آب و ہوا کیسی ہے; Intent: weather_query
Utterance: volumen på musikken der spiller nu skal op; Intent: audio_volume_up
Utterance: እባክ

## Generate prompts for each query

In [10]:
# Add a new feature column to the dataset
# Prompt: What is the intent of the following sentence?\m "{utt}"

few_shot = True
def add_prompt(example):
    if few_shot:
        example["prompt"] = f'# Detect intent of the input utterance.\n\n{prompt_exemplars}\nUtterance: {example["utt"]}; Intent:'
    else:
        example["prompt"] = f'# Detect intent of the input utterance.\n\nUtterance: {example["utt"]}; Intent:'
    
    example["str_label"] = int2str(example["intent"])
    return example


extended_eval_set = dataset['validation'].map(add_prompt)

Map: 100%|██████████| 103683/103683 [00:47<00:00, 2203.93 examples/s]


In [11]:
extended_eval_set[0]

{'id': '11',
 'locale': 'mn-MN',
 'partition': 'dev',
 'scenario': 8,
 'intent': 40,
 'utt': 'гэрлийг унтраа',
 'annot_utt': 'гэрлийг унтраа',
 'worker_id': '5',
 'slot_method': {'slot': [], 'method': []},
 'judgments': {'worker_id': ['43', '4', '30'],
  'intent_score': [1, 1, 1],
  'slots_score': [1, 1, 1],
  'grammar_score': [4, 3, 4],
  'spelling_score': [2, 2, 2],
  'language_identification': ['target', 'target', 'target']},
 'prompt': "# Detect intent of the input utterance.\n\nUtterance: የዛሬውን ቀን ልትነግሩኝ ትችላላቹ; Intent: datetime_query\nUtterance: cambia las luces a azul; Intent: iot_hue_lightchange\nUtterance: je veux un billet de train pour l' oregon; Intent: transport_ticket\nUtterance: உணவகத்தில் எடுத்துச் செல்லும் வசதி உள்ளதா; Intent: takeaway_query\nUtterance: berapakah nilai saham apple; Intent: qa_stock\nUtterance: buna; Intent: general_greet\nUtterance: olly ni matukio yapi yanafanyika karibu na mimi leo; Intent: recommendation_events\nUtterance: bu mahnını xoşlamadığımı ya

## Now let's try to predict using the LLM

In [13]:
print(extended_eval_set[0]['prompt'])

x = extended_eval_set[0]['prompt']
tok_x = tokenizer(x, return_tensors="pt")
y = model.generate(tok_x['input_ids'].to("cuda"), num_beams=5, num_return_sequences=5, max_length=2000)
print(tokenizer.decode(y[0]))

# Detect intent of the input utterance.

Utterance: የዛሬውን ቀን ልትነግሩኝ ትችላላቹ; Intent: datetime_query
Utterance: cambia las luces a azul; Intent: iot_hue_lightchange
Utterance: je veux un billet de train pour l' oregon; Intent: transport_ticket
Utterance: உணவகத்தில் எடுத்துச் செல்லும் வசதி உள்ளதா; Intent: takeaway_query
Utterance: berapakah nilai saham apple; Intent: qa_stock
Utterance: buna; Intent: general_greet
Utterance: olly ni matukio yapi yanafanyika karibu na mimi leo; Intent: recommendation_events
Utterance: bu mahnını xoşlamadığımı yadda saxla; Intent: music_dislikeness
Utterance: oprește-mi priza wemo acum; Intent: iot_wemo_off
Utterance: hvor mange grøntsager burde jeg spise om dagen; Intent: cooking_recipe
Utterance: இந்திய ரூபாவுடன் டாலரின் மாற்று விகிதம் என்ன; Intent: qa_currency
Utterance: 運輸; Intent: transport_traffic
Utterance: چطور پیش می‌رود; Intent: general_quirky
Utterance: آج آب و ہوا کیسی ہے; Intent: weather_query
Utterance: volumen på musikken der spiller nu skal o

In [3]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)
# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [4]:
tokenized_datasets['train'][0].keys()

dict_keys(['id', 'locale', 'partition', 'scenario', 'intent', 'utt', 'annot_utt', 'worker_id', 'slot_method', 'judgments', 'input_ids', 'attention_mask'])

In [5]:
# Remove columns that are not needed
to_remove = list(tokenized_datasets['train'][0].keys())

# Don't remove these columns
to_remove.remove('input_ids')
to_remove.remove('attention_mask')
to_remove.remove('intent')

tokenized_datasets = tokenized_datasets.remove_columns(to_remove)
tokenized_datasets = tokenized_datasets.rename_column('intent', 'labels')


In [6]:
tokenized_datasets['train'].features

{'labels': ClassLabel(names=['datetime_query', 'iot_hue_lightchange', 'transport_ticket', 'takeaway_query', 'qa_stock', 'general_greet', 'recommendation_events', 'music_dislikeness', 'iot_wemo_off', 'cooking_recipe', 'qa_currency', 'transport_traffic', 'general_quirky', 'weather_query', 'audio_volume_up', 'email_addcontact', 'takeaway_order', 'email_querycontact', 'iot_hue_lightup', 'recommendation_locations', 'play_audiobook', 'lists_createoradd', 'news_query', 'alarm_query', 'iot_wemo_on', 'general_joke', 'qa_definition', 'social_query', 'music_settings', 'audio_volume_other', 'calendar_remove', 'iot_hue_lightdim', 'calendar_query', 'email_sendemail', 'iot_cleaning', 'audio_volume_down', 'play_radio', 'cooking_query', 'datetime_convert', 'qa_maths', 'iot_hue_lightoff', 'iot_hue_lighton', 'transport_query', 'music_likeness', 'email_query', 'play_music', 'audio_volume_mute', 'social_post', 'alarm_set', 'qa_factoid', 'calendar_set', 'play_game', 'alarm_remove', 'lists_remove', 'transpor

## Dataloaders [Optional]

- Instead of going through individual samples in the dataset, we would like to 
batches of samples to train our model. 
- Dataloader
    1. Creates an "iterator" over the dataset, which returns a batch of samples every turn
    2. Handles shuffling
    3. Collates samples into batches, by padding the input sequences to the maximum length in the batch

In [7]:
# from torch.utils.data import DataLoader

# train_dataloader = DataLoader(
#     tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
# )
# eval_dataloader = DataLoader(
#     tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
# )

In [8]:
# Check the shape of the tensors
# for batch in tqdm(train_dataloader):
#     break
# {k: v.shape for k, v in batch.items()}

```python
{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 25]),
 'attention_mask': torch.Size([8, 25])}
```

## Load model using an `AutoModel`

In [9]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=tokenized_datasets['train'].features['labels'].num_classes)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifier.bias', 'pre_classifier.

In [10]:
# # Let's check the output of a single forward pass
# outputs = model(**batch)
# print(outputs.loss, outputs.logits.shape)

## Setup Evaluation Metric

In [11]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [12]:
# We need to define a compute_metric function that is supported by the Trainer output
# It basically converts the logits to predictions and then calls the metric
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

## Setup Trainer and Finetuning

1. When testing your code for the first time, it is a better idea to use a smaller dataset,
so that you can quickly iterate over your code.
2. We have done the same here by using 128 + 32 samples from the dataset, for training and validation respectively.

In [13]:
train_data_subset = tokenized_datasets["train"].select(range(4096))
eval_data_subset = tokenized_datasets["validation"].select(range(2096))

In [14]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=10,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    # warmup_steps=500,
    # weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    report_to="none" # set "wandb" here to log to wandb
)

**Wandb Logging**

When training ML models, keeping track of various hyperparameters, the training and validation losses, and evaluation metrics is very important.
`wandb` can help you here.

To install wandb, run following commands in your terminal:

```bash
pip install wandb
# Then login to your wandb account
wandb login
```

Once, that is done, in TrainingArguments, set the `report_to` argument to `wandb`.

In [15]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    # train_dataset=tokenized_datasets["train"],
    # eval_dataset=tokenized_datasets["validation"],
    train_dataset=train_data_subset, # Using the smaller subsets
    eval_dataset=eval_data_subset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [16]:
stats = trainer.train()

/home/bsantra/miniconda3/envs/py38/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4096
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1280
  Number of trainable parameters = 66999612
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,2.126600,4.311208,0.205153
2,1.332600,4.608765,0.259542
3,0.877300,4.879213,0.278626
4,0.628700,5.112392,0.292462
5,0.565800,5.199319,0.304389
6,0.445600,5.414936,0.302481
7,0.358800,5.228592,0.311069
8,0.224900,5.661997,0.306298
9,0.229100,5.564895,0.307252
10,0.239900,5.539486,0.310115


***** Running Evaluation *****
  Num examples = 2096
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-128
Configuration saved in ./results/checkpoint-128/config.json
Model weights saved in ./results/checkpoint-128/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-128/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-128/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 2096
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-256
Configuration saved in ./results/checkpoint-256/config.json
Model weights saved in ./results/checkpoint-256/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-256/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-256/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 2096
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-384
Configuration saved in ./results/checkpoint-384/config.json
Model w

In [17]:
# Print Stats
from pprint import pprint
pprint(trainer.evaluate())

***** Running Evaluation *****
  Num examples = 2096
  Batch size = 32


{'epoch': 10.0,
 'eval_accuracy': 0.3110687022900763,
 'eval_loss': 5.2285919189453125,
 'eval_runtime': 1.0504,
 'eval_samples_per_second': 1995.343,
 'eval_steps_per_second': 62.83}
